In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-29T16:22:37.286514-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%watermark --iversions

xgboost: 1.7.5
dask   : 2023.3.2
optuna : 3.1.1
numpy  : 1.23.5
pandas : 1.5.3
logging: 0.5.1.2



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-5c583ee7-8fb1-b26a-4bea-16e45d984a32)
GPU 1: NVIDIA H100 80GB HBM3 (UUID: GPU-a6ab06f5-a6e2-18c1-9dec-0dcd29f44a46)
GPU 2: NVIDIA H100 80GB HBM3 (UUID: GPU-bb8d5098-3c56-c48d-a0a3-fcdfcec6d3f5)
GPU 3: NVIDIA H100 80GB HBM3 (UUID: GPU-cdbe686b-1611-999b-8e8d-a3c5f35b40c4)
GPU 4: NVIDIA H100 80GB HBM3 (UUID: GPU-0df1cef0-fc95-cc88-b5f4-239889b3acba)
GPU 5: NVIDIA H100 80GB HBM3 (UUID: GPU-9dca657c-dbe4-08b7-fe7c-5653f183f0b6)
GPU 6: NVIDIA H100 80GB HBM3 (UUID: GPU-33389782-e2ad-5022-997d-cf470313879c)
GPU 7: NVIDIA H100 80GB HBM3 (UUID: GPU-926eaa05-87fb-35e0-0a98-adb2ad41d0be)


In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-29 16:22:45,277 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-8e1nuff3', purging
2023-05-29 16:22:45,278 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vuzyqtwv', purging
2023-05-29 16:22:45,278 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-29 16:22:45,278 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-29 16:22:45,278 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ad2cc3ro', purging
2023-05-29 16:22:45,278 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-vjlckzo_', purging
2023-05-29 16:22:45,278 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-29 16:22:45,278 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-29 16:22:45,278 - dist

In [7]:
!nvidia-smi

Mon May 29 16:22:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 80G...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   32C    P0   110W / 700W |    525MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA H100 80G...  On   | 00000000:43:00.0 Off |                    0 |
| N/A   

In [8]:
%%time
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input2/xgtrain_fold_{i}_l.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input2/xgval_fold_{i}_l.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4
CPU times: user 713 ms, sys: 905 ms, total: 1.62 s
Wall time: 19.2 s


In [9]:
train = pd.read_csv('../input/train.csv.zip')

shift = 200

target0 = train['loss'].values
target = np.log(target0+shift)

In [10]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con =2
    x =preds-labels
    grad =con*x / (np.abs(x)+con)
    hess =con**2 / (np.abs(x)+con)**2
    return grad, hess 


def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [11]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        #'objective': 'reg:squarederror', 
        'base_score':7.76,
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_boost_round=num_round, obj=logregobj)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = np.exp(predictions) - shift
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target0, train_oof)
    
    return mae

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_l_6.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_l_6.db", study_name="five_fold_optuna_xgb_l_6", direction='minimize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/tmp/ipykernel_360162/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:14: FutureWarning: sugge

CPU times: user 32.5 s, sys: 30.1 s, total: 1min 2s
Wall time: 7min 36s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1243.018179,0.024561,0.430946,rmse,2.297308,1.691877,0.001718,15,98,0.898109,COMPLETE
1,1,1163.725316,0.450500,0.629448,rmse,0.223763,0.445528,0.004066,9,69,0.761649,COMPLETE
2,2,1164.384459,0.035761,0.602513,rmse,0.027854,0.002979,0.089153,10,118,0.965994,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_6.csv', index=False)
df

/tmp/ipykernel_360162/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:14: FutureWarning: sugge

CPU times: user 26.7 s, sys: 27 s, total: 53.8 s
Wall time: 9min 12s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1243.018179,0.024561,0.430946,rmse,2.297308,1.691877,0.001718,15,98,0.898109,COMPLETE
1,1,1163.725316,0.450500,0.629448,rmse,0.223763,0.445528,0.004066,9,69,0.761649,COMPLETE
2,2,1164.384459,0.035761,0.602513,rmse,0.027854,0.002979,0.089153,10,118,0.965994,COMPLETE
3,3,1141.185647,0.229325,0.863084,rmse,2.248218,0.013372,0.019631,23,293,0.863903,COMPLETE
4,4,1143.885455,0.032164,0.939177,rmse,0.006129,0.013264,0.037778,5,4,0.643277,COMPLETE
5,5,1148.891235,0.038744,0.405729,rmse,4.604734,0.800343,0.023820,6,222,0.814676,COMPLETE
6,6,1146.297831,0.537314,0.307181,rmse,6.492149,8.115651,0.019032,16,1,0.647008,COMPLETE
7,7,1139.728026,1.596728,0.769397,rmse,0.001316,2.113918,0.008295,18,183,0.831305,COMPLETE


In [17]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_6.csv', index=False)
df.head(30)

/tmp/ipykernel_360162/481481954.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_360162/481481954.py:14: FutureWarning: sugge

CPU times: user 11min 11s, sys: 10min 11s, total: 21min 23s
Wall time: 3h 49min 55s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1243.018179,0.024561,0.430946,rmse,2.297308,1.691877,0.001718,15,98,0.898109,COMPLETE
1,1,1163.725316,0.450500,0.629448,rmse,0.223763,0.445528,0.004066,9,69,0.761649,COMPLETE
2,2,1164.384459,0.035761,0.602513,rmse,0.027854,0.002979,0.089153,10,118,0.965994,COMPLETE
3,3,1141.185647,0.229325,0.863084,rmse,2.248218,0.013372,0.019631,23,293,0.863903,COMPLETE
4,4,1143.885455,0.032164,0.939177,rmse,0.006129,0.013264,0.037778,5,4,0.643277,COMPLETE
5,5,1148.891235,0.038744,0.405729,rmse,4.604734,0.800343,0.023820,6,222,0.814676,COMPLETE
6,6,1146.297831,0.537314,0.307181,rmse,6.492149,8.115651,0.019032,16,1,0.647008,COMPLETE
7,7,1139.728026,1.596728,0.769397,rmse,0.001316,2.113918,0.008295,18,183,0.831305,COMPLETE
8,8,1171.634198,0.005621,0.456363,rmse,0.039913,2.331744,0.005835,6,28,0.494178,COMPLETE
9,9,1441.666180,0.006999,0.653651,rmse,1.100680,0.003048,0.001372,3,90,0.491001,COMPLETE


In [ ]:
df

In [ ]:
df.value.min()